```{=latex}
\usepackage{hyperref}
\usepackage{graphicx}
\usepackage{listings}
\usepackage{textcomp}
\usepackage{fancyvrb}

\newcommand{\passthrough}[1]{\lstset{mathescape=false}#1\lstset{mathescape=true}}
\newcommand{\tightlist}{}
```

```{=latex}
\title{Unit Testing Your Web Application}
\author{Moshe Zadka -- https://cobordism.com}
\date{}

\begin{document}
\begin{titlepage}
\maketitle
\end{titlepage}

\frame{\titlepage}
```

```{=latex}
\begin{frame}
\frametitle{Acknowledgement of Country}

Belmont (in San Francisco Bay Area Peninsula)

Ancestral homeland of the Ramaytush Ohlone people

\end{frame}
```

I live in Belmont,
in the San Francisco Bay Area Peninsula.
I wish to acknowledge it as the
ancestral homeland
of the
Ramaytush Ohlone people.

In [124]:
import json
import pyramid.response
import pyramid.config
import httpx
from unittest import mock
from hamcrest import assert_that, equal_to
import io

```{=latex}
\begin{frame}
\frametitle{Outline}

Intro to Pyramid \pause

What are Unit tests? \pause

What is httpx? \pause

Unit test examples! \pause

\end{frame}
```

I'll be giving a quick into to Pyramid. Then we'll talk a little about what unit tests are. We will go over httpx, which is a useful tool for writing unit tests for WSGI apps. Finally, the reason we're all here -- actually write some unit tests!

## Pyramid Example

```{=latex}
\begin{frame}
\frametitle{Crash Course in Pyramid}

Some quick examples!

\end{frame}
```

A full tutorial on Pyramid would be a full semester.
There is no chance to do this here.
This will cover just enough of Pyramid to give a foundation
for the later examples.

### JSON Renderer

```{=latex}
\begin{frame}[fragile]
\frametitle{JSON app: View Function}
```

In [165]:
def jsonv(request):
    return {}

```{=latex}
\end{frame}
```

This function takes a request,
and returns an empty dictionary.
It is a one-line
"view"
function.

```{=latex}
\begin{frame}[fragile]
\frametitle{JSON app: Configurator}
```

In [165]:
with pyramid.config.Configurator() as config:
    config.add_route("json", "/json")
    config.add_view(jsonv, route_name="json",
                    renderer="json")
    

```{=latex}
\end{frame}
```

The
`Configurator`
maps the function to a route.
It does so by defining a
route
based on a path,
and then attach the function to the route.

```{=latex}
\begin{frame}[fragile]
\frametitle{JSON app: WSGI app}
```

In [165]:
app = config.make_wsgi_app()

```{=latex}
\end{frame}
```

Finally,
the
WSGI
application is generated.

This is almost the simplest Pyramid APP that you can write:
it has one route and one view.

In [166]:
client = httpx.Client(app=app, base_url="https://example.com/")

```{=latex}
\begin{frame}[fragile]
\frametitle{JSON View Retrieval}
```

In [167]:
request = client.get("/json")
request.json()

{}

```{=latex}
\end{frame}
```

The app can be run with any
WSGI server.
After running it,
you can grab the URL
(`/json`).

The result is
`"{}"`,
which the client's
`.json()`
method transforms into an empty Python dictionary.

### Using Route Matches

```{=latex}
\begin{frame}[fragile]
\frametitle{Matcher View}
```

In [342]:
def matches(request):
    return dict(name=request.matchdict["name"])
with pyramid.config.Configurator() as config:
    config.add_route("matches", "/matches/{name}")
    config.add_view(matches, route_name="matches",
                    renderer="json")
app = config.make_wsgi_app()

```{=latex}
\end{frame}
```

Up the sophistication ladder a little,
this route does something interesting.
The route
*matches*
a part of the path.

The matched part of the path is in the
`request.matchdict`.
The return value from the request can use this value.

In [343]:
client = httpx.Client(app=app, base_url="https://example.com/")

```{=latex}
\begin{frame}[fragile]
\frametitle{Matcher View Retrieval}
```

In [344]:
request = client.get("/matches/hello")
request.json()

{'name': 'hello'}

```{=latex}
\end{frame}
```

When accessing the app,
and passing the
`hello`
string as part of the path,
the response contains the name.

### Using Settings

```{=latex}
\begin{frame}[fragile]
\frametitle{Settings View}
```

In [171]:
def setting(request):
    return dict(
        field=request.registry.settings["field"]
    )
with pyramid.config.Configurator(
    settings=dict(field="field_value")
) as config:
    config.add_route("setting", "/setting")
    config.add_view(setting, route_name="setting",
                    renderer="json")
app = config.make_wsgi_app()

```{=latex}
\end{frame}
```

Another way to make Pyramid applications more sophisiticated
is to use the
`settings`
parameter to the configurator.
This parameter allows passing in settings data:
tweaking the configuration,
globally-available objects,
and the like.

In this example,
the settings passes in a
`field`
parameter.
This parameter can be accessed from a view function via the
`request.registry.settings`
parameter.

In [172]:
client = httpx.Client(app=app, base_url="https://example.com/")

```{=latex}
\begin{frame}[fragile]
\frametitle{Settings View Retrieval}
```

In [173]:
request = client.get("/setting")
request.json()

{'field': 'field_value'}

```{=latex}
\end{frame}
```

Retrieving the relevant URL
returns the passed-in setting.

### Pyramid crash course review

```{=latex}
\begin{frame}[fragile]
\frametitle{View Function}
```

In [346]:
def lookup(
    request: pyramid.request.Request
) -> str:
#    ^^^
#    Will be jsonified
    matcher = request.registry.settings["matcher"] 
#             ^^^^^^^^^^^^^^^^^^^^^^^^^
#             Access settings through request
    name = request.matchdict["name"]
#          ^^^^^^^^^^^^^^^^^
#          Get parameter from route
    return matcher.get(name.lower(), "default")
#          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
#          Business logic

```{=latex}
\end{frame}
```

As a way of summary,
this is an application
that has everything covered so far.

The view uses both the
*settings*
and the
*matchdict*
to return a value.

```{=latex}
\begin{frame}[fragile]
\frametitle{Routing}
```

In [347]:
def include_lookup(config):
    config.add_route("lookup", "/lookup/{name}")
#          ^^^^^^^^^|^^^^^^^   ^^^^^^^^^^^^^^^
#          add a new|route    route path
#          route    |name
    config.add_view(lookup, route_name="lookup",
#          ^^^^^^^^|^^^^^   ^^^^^^^^^^^^^^^^^^^
#          add a   |view    route to attach      
#          view to |callable
#          a route |
                    renderer="json")
#                   ^^^^^^^^^^^^^^^
#                   Convert to a JSON response

```{=latex}
\end{frame}
```

This function adds a route,
and a matching view,
to a configurator.
This has been broken into a function both to keep
each piece of code separate to allow over-explaining it,
but also because this is not an uncommon pattern in Pyramid.

```{=latex}
\begin{frame}[fragile]
\frametitle{Settings}
```

In [348]:
settings = dict(
#^^     
#settings      
    matcher=dict(special="hello"),
#   ^^^^^^^ ^^^^^^^^^^^^^^^^^^^^^^
#   settings ^^^^^^^^^^^^^^^^^^^^^
#   key       Mapping "special"
#             name to "hello"
)

```{=latex}
\end{frame}
```

The setting dictionary maps the key
`matcher`
to a dictionary with how different paths should be treated.
Recall that without
"special"
handling,
the value returned is the default.

```{=latex}
\begin{frame}[fragile]
\frametitle{Configuration}
```

In [349]:
with pyramid.config.Configurator(
#^^^ ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
#Commit at the end      ^^^^^^^^^^
#                       Configurator
#                       class
    settings=settings,
#   ^^^^^^^  ^^^^^^^^
#   settings pre-configured
#            dict
) as config:
#    ^^^^^^
#    configurator
#    object
    include_lookup(config)

```{=latex}
\end{frame}
```

The
`Configurator`
combines the settings
and the routing/view configuration
into one thing.

```{=latex}
\begin{frame}[fragile]
\frametitle{App creation}
```

In [350]:
app = config.make_wsgi_app(
#                 ^^^^
#         Web Standard Gateway Interface
)

```{=latex}
\end{frame}
```

Finally,
the application is created.
It is now possible to serve this application.

In [351]:
client = httpx.Client(app=app, base_url="https://example.com/")
request = client.get("/lookup/SPECIAL")
request.json()

'hello'

## Unit Testing

### What is a unit test?

```{=latex}
\begin{frame}
\frametitle{Unit test: rough definition}

\pause

Runs the code

\pause

Might fail on buggy code

\pause

Self-contained

\end{frame}
```

Unit tests are defined differently by different people.
"What is a unit test" is a question that sparks frequent arguments.

For our purposes, this will be the version we use.
The most important part is
"self-contained".

This means that it should not do any network traffic,
or rely on the machine being in a specific state.
This is not the usual definition,
which is more specific,
but it will be enough for now.

### What is a mock?

```{=latex}
\begin{frame}
\frametitle{Mock}

\pause

Fake object

\pause

Configurable behavior

\pause

Records access

\end{frame}
```

Often,
in order to be
*self-contained*,
a unit test will use a
fake object.
Mocks,
in Python,
are good tools to build such fake objects.

They have configurable behavior:
different ways to have them respond to methods.
They also record method calls.

### Goal of unit test

```{=latex}
\begin{frame}
\frametitle{Good Unit Test}

\pause

Avoid failing on valid: \pause

Public APIs \pause

Reduce assumptions
\end{frame}
```

A unit test should not fail on valid code.
This is not always as possible,
and not as easy as it seems!

The unit test might make calls to internal APIs,
which can change.
This leaves the code valid but breaks the unit test.

The unit test might have assumptions that are not part of the definition
of the API.
For example,
it might be assuming an order of the returned values.

### Patching as assumption

```{=latex}
\begin{frame}
\frametitle{Patch Makes Bad Unit Test}

\pause

Patch: Temporarily replacing a global

\pause

Assumption: Global used

\pause

Assumption: Used through path

\end{frame}
```

One assumption that unit tests often make is to assume
a specific global being used.
This is called
"patching",
and is popular advice.

For example,
patching the
"random.random"
function to return a specific value.

This makes two assumptions:

* The random value is generated by `random.random`
  (and not, say, `random.uniform`).
* The function is accessed as `random.random`
  and not via "from random import random".

The test will fail if either of these two assumptions are violated.
This includes the case where there are no new bugs being introduced.

### Parts of unit test

```{=latex}
\begin{frame}
\frametitle{Unit test anatomy}

\pause

Set-up

\pause

Execution

\pause

Post-process \pause
(optional) \pause

Verification

\end{frame}
```

All tests,
and specifically unit tests,
can be thought of as containing four parts:

* Set up: Defining variables or creating objects the unit needs to run.
* Execution: Calling the unit.
* Post-process: Taking the outputs from the unit and doing some processing on it.
* Verification: Checking the post-processed results are as expected.

Some divisions combine the
"post process"
and
"verification"
steps.
It is useful to separate them to understand unit tests better.

## Python Web Concepts

### WSGI

```{=latex}
\begin{frame}[fragile]
\frametitle{WSGI}

Framework \pause

to server

\end{frame}
```

WSGI is a standard for how a
*framework*,
like Pyramid or Django,
can be served by a server,
like
Gunicorn
or
uwsgi.

### `httpx`

```{=latex}
\begin{frame}[fragile]
\frametitle{httpx}

Requests alternative \pause

Can consume WSGI directly
\end{frame}
```

httpx is a requests alternative.
One of the nice features it has is
that is can be directed to consume WSGI.

In other words, instead of
Framework to server to TCP port to HTTP client,
this makes
HTTP client to framework.
Much easier for testing...hmmmm.....

```{=latex}
\begin{frame}[fragile]
\frametitle{httpx with WSGI}

Creating the client: \pause
```

In [281]:
client = httpx.Client(
    base_url="https://example.com/",
    app=app,
)

```{=latex}
\end{frame}
```

This creates a client that points to a
WSGI
application.

```{=latex}
\begin{frame}[fragile]
\frametitle{httpx with WSGI}

Calling the client: \pause

```

In [353]:
response = client.get("/lookup/SPECIAL")

```{=latex}
\end{frame}
```

Calling a method on the client returns a
"response"
object.

```{=latex}
\begin{frame}[fragile]
\frametitle{httpx with WSGI}

Using the httpx response object: \pause

```

In [354]:
response.json()

'hello'

```{=latex}
\end{frame}
```

Asking the response to get the
JSON-parsed
version of the body
yields the expected output.

## Unit test examples!

### System under test

```{=latex}
\begin{frame}[fragile]
\frametitle{System Under Test: View}
```

In [314]:
def lookup(
    request: pyramid.request.Request
) -> str:
    matcher = request.registry.settings["matcher"] 
    name = request.matchdict["name"]
    return matcher.get(name, "default")
#                      ^^^^^
#                      Bug: missing .lower()

```{=latex}
\end{frame}
```

In unit tests,
the
"system under test"
refers to the
"unit":
the code that needs to be tested.
This system has a bug,
so unit tests can fail.

```{=latex}
\begin{frame}[fragile]
\frametitle{System Under Test: App}
```

In [340]:
def make_app(special_value):
#^^          ^^^^^^^^^^^^^
#^^          Parameter for
#            the application
#Creating the app from a function,
#not at top level
    settings = dict(
        matcher=dict(special=special_value),
    )
    with pyramid.config.Configurator(
        settings=settings
    ) as config:
        include_lookup(config)
    return config.make_wsgi_app()

```{=latex}
\end{frame}
```

The system also includes a function to create the app.
This makes it easier to write unit tests:
they can call the function explicitly to create an app
with the right settings.

### Mocking Request

```{=latex}
\begin{frame}[fragile]
\frametitle{Calling function directly}

Setup:

```

In [316]:
request = mock.MagicMock()
request.registry.settings = dict(
    matcher=dict(special="hello"),
)
request.matchdict = dict(name="SPECIAL")

```{=latex}
\end{frame}
```

The first way to test is to call the view function directly.
This is sometimes the best way,
since it can avoid the routing and app creation.
In this example,
it does not save too much code.

```{=latex}
\begin{frame}[fragile]
\frametitle{Calling function directly}

Execute:
```

In [317]:
result = lookup(request)

```{=latex}
\end{frame}
```

The execution is remarkably short:
calling the function on the set-up object.
This is nice and simple!

```{=latex}
\begin{frame}[fragile]
\frametitle{Calling function directly}

Verification
```

In [318]:
try:
    assert_that(result, equal_to("hello"))
except AssertionError as exc:
    print(exc)


Expected: 'hello'
     but: was 'default'



```{=latex}
\end{frame}
```

For verification,
this test uses
`hamcrest`.
It expects the
`hello`
response,
and gets the
`default`
one.
The test found the bug,
hooray!

### Calling WSGI

```{=latex}
\begin{frame}[fragile]
\frametitle{Calling WSGI}

Setup:
```

In [319]:
environ = dict(
    PATH_INFO="/lookup/SPECIAL",
    REQUEST_METHOD="GET",
)
start_response = mock.MagicMock(
    return_value=io.BytesIO()
)

```{=latex}
\end{frame}
```

The other layer to test this is to go through
WSGI.
The environment
and
`start_response`
are part of the WSGI standard.

```{=latex}
\begin{frame}[fragile]
\frametitle{Calling WSGI}

Execute:
```

In [320]:
app = make_app("hello")
base_parts = app(environ, start_response)

```{=latex}
\end{frame}
```

This time the execution involves calling the
`make_app`
function,
and then calling the result on the objects from the
set up phase.

```{=latex}
\begin{frame}[fragile]
\frametitle{Calling WSGI}

Post-process:
```

In [321]:
parts = [start_response.return_value.getvalue()]
parts.extend(base_parts)
args, kwargs = start_response.call_args
status, headers = args
result = json.loads(b"".join(parts).decode("utf-8"))

```{=latex}
\end{frame}
```

Post-processing is kind of awkward.
There are two ways for WSGI apps to
return a response body:
using the
`start_response`
return value,
and returning them directly from the function.

This adds everything up and parses the JSON.

```{=latex}
\begin{frame}[fragile]
\frametitle{Calling WSGI}

Verification
```

In [322]:
try:
    assert_that(result, equal_to("hello"))
except AssertionError as exc:
    print(exc)


Expected: 'hello'
     but: was 'default'



```{=latex}
\end{frame}
```

The verification is the same:
comparing the JSON-decoded result.

### Using `httpx`

```{=latex}
\begin{frame}[fragile]
\frametitle{Using httpx}

Setup:
```

In [323]:
app = make_app("hello")
client = httpx.Client(
    base_url="https://example.com/",
    app=app,
)

```{=latex}
\end{frame}
```

The first version had a problem of calling a non-public API:
if the name of
`lookup`
is changed,
and then is changed in
`make_app`,
then the result is valid but the test fails.

The second version had a different problem.
Both set-up and post-processing were awkward.

Using
`httpx`
can use the best of all worlds.
The set-up is more straightforward.


```{=latex}
\begin{frame}[fragile]
\frametitle{Using httpx}

Execute:
```

In [324]:
resp = client.get("/lookup/SPECIAL")

```{=latex}
\end{frame}
```

The execution is the same as retrieving the result
from a real server.

```{=latex}
\begin{frame}[fragile]
\frametitle{Using httpx}

Post-process:
```

In [325]:
result = resp.json()

```{=latex}
\end{frame}
```

Post-processing is easier.
One method call on the response object
to get the JSON-parsed body.

```{=latex}
\begin{frame}[fragile]
\frametitle{Using httpx}

Verification:
```

In [326]:
try:
    assert_that(result, equal_to("hello"))
except AssertionError as exc:
    print(exc)


Expected: 'hello'
     but: was 'default'



```{=latex}
\end{frame}
```

Verification,
once again,
is the same.

This tests the
*public*
API:
`make_app`
returns a WSGI app
which responds to
`/location/SPECIAL`
with
`hello`.


## Summary

### Pyramid: Concepts

```{=latex}
\begin{frame}
\frametitle{Summary: Pyramid}

\pause

View: \pause
request, response, route \pause

Route: \pause
URL match \pause

Registry: \pause
Parameters, shared objects \pause

Configurator: \pause
Views, Routes, Registry \pause

WSGI: \pause
Configurator to server \pause
(or httpx!)

\end{frame}
```

Pyramid maps views to routes.
The views can use the registry.

The configurator takes all of these
and returns a WSGI app
that can be used for HTTP servers
or
`httpx`.

### Unit Testing

```{=latex}
\begin{frame}
\frametitle{Summary: Unit Test}

\pause

Verify code, \pause
quickly, \pause
and safely: \pause

Mock! \pause

Pre-plan \pause

Prefer stable APIs

\end{frame}
```

Unit tests verify code quickly and safely.
They use mock to prevent the code from having system dependencies.

Pre-planning the API
makes a more testable API

## Bonus material

```{=latex}
\begin{frame}
\frametitle{Bonus}

Extra! Extra!
\end{frame}
```

### Pyramid Concepts

#### View

```{=latex}
\begin{frame}
\frametitle{View}

Function \pause

Argument: Request \pause

Return value: Response

\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{View Route}

Route \pause

to View \pause

\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{View Route Predicates}

Route to view can be conditional:\pause

Request type,\pause

Content type,\pause

Arbitrary predicates

\end{frame}
```

#### Route

```{=latex}
\begin{frame}
\frametitle{Route}

Path \pause

to route name \pause

\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{Route: advanced}

\pause

Path fragment matching \pause

...and more

\end{frame}
```

#### Registry

```{=latex}
\begin{frame}
\frametitle{Registry: Application Parameters}

\pause

Settings: ad-hoc dictionary

\end{frame}
```

#### Configurator

```{=latex}
\begin{frame}[fragile]
\frametitle{Configurator}

Routes \pause

Views \pause

Registry \pause

\end{frame}
```

### Routing extras

#### Static view

```{=latex}
\begin{frame}[fragile]
\frametitle{Static Value}
```

In [310]:
def empty(request):
    return pyramid.response.Response(
        json.dumps({}).encode("ascii"),
        content_type="application/json",
    )
with pyramid.config.Configurator() as config:
    config.add_route("root", "/")
    config.add_view(empty, route_name="root")
app = config.make_wsgi_app()

```{=latex}
\end{frame}
```

In [311]:
client = httpx.Client(app=app, base_url="https://example.com/")

```{=latex}
\begin{frame}[fragile]
\frametitle{Static View Retrieval}
```

In [312]:
request = client.get("/")
request.json()

{}

```{=latex}
\end{frame}
```

#### Using Parameters

```{=latex}
\begin{frame}[fragile]
\frametitle{Parameter View}
```

In [313]:
def params(request):
    return dict(thing=request.params["thing"])
with pyramid.config.Configurator() as config:
    config.add_route("params", "/params")
    config.add_view(params, route_name="params", renderer="json")
app = config.make_wsgi_app()

```{=latex}
\end{frame}
```

In [299]:
client = httpx.Client(app=app, base_url="https://example.com/")

```{=latex}
\begin{frame}[fragile]
\frametitle{Parameter View Retrieval}
```

In [300]:
request = client.get("/params?thing=hello")
request.json()

{'thing': 'hello'}

```{=latex}
\end{frame}
```

#### Using Request Body

```{=latex}
\begin{frame}[fragile]
\frametitle{Body View}
```

In [301]:
def body(request):
    return dict(field=request.json_body["field"])
with pyramid.config.Configurator() as config:
    config.add_route("body", "/body")
    config.add_view(body, route_name="body", renderer="json")
app = config.make_wsgi_app()

```{=latex}
\end{frame}
```

In [302]:
client = httpx.Client(app=app, base_url="https://example.com/")

```{=latex}
\begin{frame}[fragile]
\frametitle{Body View Retrieval}
```

In [303]:
request = client.post("/body", json=dict(field="hello"))
request.json()

{'field': 'hello'}

```{=latex}
\end{frame}
```

### Registry extras

```{=latex}
\begin{frame}
\frametitle{Registry: Application Parameters}

\pause

Utility: Scalable configuration


\end{frame}
```

In [304]:
import zope.interface
import attrs

```{=latex}
\begin{frame}[fragile]
\frametitle{Interfaces and Components}
```

In [305]:
class IValueGetter(zope.interface.Interface):
    def get_value(self) -> int:
        ...

@zope.interface.implementer(IValueGetter)
@attrs.frozen
class ValueStorer:
    _value: int
    
    def get_value(self):
        return self._value

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{Interface-based Registry}
```

In [306]:
config = pyramid.config.Configurator()
config.registry.registerUtility(ValueStorer(value=42))
config.registry.getUtility(IValueGetter).get_value()

42

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{Why Interface-based Registry?}

Namespaced\pause

Semantically meaningful \pause

Configuration

\end{frame}
```

### Configurator extras

```{=latex}
\begin{frame}[fragile]
\frametitle{Configurator Inclusion}
```

In [307]:
def root_stuff(config):
    config.add_route("root", "/")
    config.add_view(empty, route_name="root")    
with pyramid.config.Configurator() as config:
    config.include(root_stuff)
    app = config.make_wsgi_app()

In [308]:
client = httpx.Client(app=app, base_url="https://example.com/")

In [309]:
request = client.get("/")
request.json()

{}

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{Configurator Inclusion}

Include allows: \pause

\begin{itemize}
\item Callable \pause
\item Callable dotted name \pause
\item Module (will use "includeme") \pause
\item Module dotted name
\end{itemize}

\end{frame}
```

### Scanning

```{=latex}
\begin{frame}[fragile]
\frametitle{Configurator Scanning}

* Scan a package
* Find functions decorated "view config"

\end{frame}
```

```{=latex}
\end{document}
```